# Laboratorio no. 5 Threat Hunting
Marco Jurado 20308

In [35]:
#pip install python-whois

In [36]:
import pandas as pd
import json
import whois

## Parte 1

Abrimos large_eve.json y luego verificamos tener los elementos correctos

In [37]:
json_file_path = 'large_eve.json'
data = []

with open(json_file_path, 'r') as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en la línea: {e}")

print(f'Total de registros cargados: {len(data)}')

df = pd.DataFrame(data)
print(df.info())

Total de registros cargados: 746909
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746909 entries, 0 to 746908
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   timestamp     746909 non-null  object 
 1   flow_id       746892 non-null  float64
 2   pcap_cnt      189115 non-null  float64
 3   event_type    746909 non-null  object 
 4   vlan          188065 non-null  float64
 5   src_ip        746892 non-null  object 
 6   src_port      746278 non-null  float64
 7   dest_ip       746892 non-null  object 
 8   dest_port     746278 non-null  float64
 9   proto         746892 non-null  object 
 10  alert         2145 non-null    object 
 11  ssh           6451 non-null    object 
 12  dns           15749 non-null   object 
 13  tx_id         88186 non-null   float64
 14  http          170981 non-null  object 
 15  app_proto     186205 non-null  object 
 16  fileinfo      82787 non-null   object 
 17  tls         

Filtramos DNS y verificamos tener exito al tener los elementos.

In [38]:
dns_records = [record for record in data if record.get('event_type') == 'dns']
print(f'Total de registros DNS: {len(dns_records)}')
df_dns = pd.json_normalize(dns_records)
print(f'Shape del DataFrame: {df_dns.shape}')
print(df_dns.head())
df_dns_a_records = df_dns[df_dns['dns.rrtype'] == 'A']
print(df_dns_a_records.sample(2))

Total de registros DNS: 15749
Shape del DataFrame: (15749, 18)
                         timestamp           flow_id  pcap_cnt event_type  \
0  2017-07-22T17:33:16.661646-0500  1327836194150542     22269        dns   
1  2017-07-22T17:33:24.990320-0500  2022925111925872     54352        dns   
2  2017-07-22T17:33:27.379891-0500   578544790391795     54519        dns   
3  2017-07-22T17:33:27.380146-0500   578544790391795     54520        dns   
4  2017-07-22T17:33:27.380146-0500   578544790391795     54520        dns   

   vlan                                   src_ip  src_port  \
0   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     59680   
1   110  2001:0dbb:0c18:0011:0260:6eff:fe30:0863     38051   
2   150                          192.168.205.170     31393   
3   150                            192.168.207.4        53   
4   150                            192.168.207.4        53   

                                   dest_ip  dest_port proto dns.type  dns.id  \
0  2001:0500:0001:000

Mostramos dos elementos cualquiera

In [39]:
print(json.dumps(dns_records[0], indent=2))
print(json.dumps(dns_records[1], indent=2))

{
  "timestamp": "2017-07-22T17:33:16.661646-0500",
  "flow_id": 1327836194150542,
  "pcap_cnt": 22269,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 59680,
  "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 15529,
    "rrname": "api.wunderground.com",
    "rrtype": "A",
    "tx_id": 0
  }
}
{
  "timestamp": "2017-07-22T17:33:24.990320-0500",
  "flow_id": 2022925111925872,
  "pcap_cnt": 54352,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 38051,
  "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 58278,
    "rrname": "stork79.dropbox.com",
    "rrtype": "A",
    "tx_id": 0
  }
}


Mostramos la forma de DNS

In [40]:
print(df_dns.shape)

(15749, 18)


filtrar los registros DNS para aquellos registros tipo A

In [41]:
print(f"Total de registros tipo 'A': {len(df_dns_a_records)}")

Total de registros tipo 'A': 2849


Filtrar los dominios únicos

In [42]:
unique_a_records = df_dns_a_records['dns.rrname'].unique()
print(f"Total de dominios únicos tipo 'A': {len(unique_a_records)}")


Total de dominios únicos tipo 'A': 177


Función que extraiga el TLD de un dominio completo

In [43]:
from urllib.parse import urlparse

def get_tld(domain):
    if domain:
        if not domain.startswith(('http://', 'https://')):
            domain = 'http://' + domain
        tld = urlparse(domain).hostname.split('.')[-2:]
        return '.'.join(tld) if len(tld) == 2 else domain
    return ""

print(get_tld("api.wunderground.com"))
print(get_tld("safebrowsing.clients.google.com.home"))


wunderground.com
com.home


In [44]:
df_unique_a = pd.DataFrame(unique_a_records, columns=['domain'])

df_unique_a['domain_tld'] = df_unique_a['domain'].apply(get_tld)

df_unique_a = df_unique_a.drop(columns=['domain'])

print(df_unique_a)

           domain_tld
0    wunderground.com
1         dropbox.com
2           aoltw.net
3            com.home
4         mozilla.com
..                ...
172   com.localdomain
173     microsoft.com
174      verisign.com
175            22.201
176          real.com

[177 rows x 1 columns]


## Parte 2

Primero vamos a realizar la importación del clasificador adjuntado en el proyecto.

In [45]:
from clasificador import *

Iniciamos el proceso de filtrar aquellos considerados como DGA (valor 1 ).

In [46]:
df_classified = clasificacion(df_unique_a)
df_dga = df_classified[df_classified['isDGA'] == 1]
df_dga_unique = df_dga.drop_duplicates(subset=['domain_tld'])
print(df_dga_unique)
print(f'Total de dominios únicos considerados DGA: {len(df_dga_unique)}')

                    domain_tld  isDGA
1                  dropbox.com      1
2                    aoltw.net      1
5               metasploit.com      1
18            22.110phpmyadmin      1
24   110phpmyadmin.localdomain      1
25                 windows.com      1
27               ntkrnlpa.info      1
34              sql-ledger.org      1
35         backtrack-linux.org      1
43              hackerlabs.vpn      1
48              phpmyadmin.net      1
51               microsoft.com      1
60              postgresql.org      1
62                 freepbx.org      1
64           bigflickrfeed.com      1
66                 http://wpad      1
78                  flickr.com      1
82             malwarecity.com      1
83                facebook.com      1
87                  xmarks.com      1
89                 cakephp.org      1
92                   adams.net      1
95                clarkson.edu      1
97                   ecvps.com      1
98                    wisc.edu      1
100         

C:\Users\marco\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Parte 3

Creación de función con el promt de chatGPT:

"Quiero escribir una función que sea capaz de tomar un listado de elementos de tipo dominios considerados como sospechosos. Vas a utilizar una lista de un millón de TLD y para esta función debes de devolver el valor '0' si el TLD que evaluarás se encuentra en el listado y '1' si no está."

In [47]:
top_1m_file_path = 'top-1m.csv'
with open(top_1m_file_path, 'r') as file:
    top_1m_tlds = set(line.strip().split(',')[1] for line in file)

def check_tld_in_list(tld):
    """
    Devuelve 0 si el TLD se encuentra en la lista de un millón de TLDs,
    y 1 si no está.
    """
    return 0 if tld in top_1m_tlds else 1

In [48]:
df_dga_unique = df_dga_unique.copy()

df_dga_unique.loc[:, 'in_known_list'] = df_dga_unique['domain_tld'].apply(check_tld_in_list)

df_suspected_domains = df_dga_unique[df_dga_unique['in_known_list'] == 1]

df_suspected_domains = df_suspected_domains.drop_duplicates(subset=['domain_tld'])
print(f'Total de dominios sospechosos únicos: {len(df_suspected_domains)}')


Total de dominios sospechosos únicos: 17


Creamos una función basada en el TLD para la fecha de creación

In [49]:
def obtener_fecha_creacion(tld):
    try:
        dominio_whois = whois.whois(tld)
        return dominio_whois.creation_date
    except Exception as e:
        return f"Error al obtener la fecha: {e}"

In [51]:
df_suspected_domains['fecha_creacion'] = df_suspected_domains['domain_tld'].apply(obtener_fecha_creacion)

In [54]:
df_suspected_domains[['domain_tld', 'fecha_creacion']]

,domain_tld,fecha_creacion
2,aoltw.net,2000-01-10 04:09:19
18,22.110phpmyadmin,None
24,110phpmyadmin.localdomain,None
27,ntkrnlpa.info,2023-06-27 08:18:55
34,sql-ledger.org,2000-09-08 16:25:09
35,backtrack-linux.org,2009-04-29 00:26:42
43,hackerlabs.vpn,None
64,bigflickrfeed.com,"Error al obtener la fecha: No match for ""BIGFL..."
66,http://wpad,None
82,malwarecity.com,"[2008-02-06 15:42:35, 2008-02-06 00:00:00]"


Ahora vamos a ver los elementos que tenemos en la tabla y al ojo crítico vamos a determinar cuales de estos se ven sospechosos según criterios que hemos visto en clase y en los laboratorios pasados.

- ejfodfmfxlkgifuf.xyz
- vtlfccmfxlkgifuf.com
- cakephp.org
- 110phpmyadmin.localdomain
- 22.110phpmyadmin
- malwarecity.com